# Basis-Function Meta-Learaning for Rapid Introspective Neural Adaptation (RINA)

This script loads quadrotor flight data in different wind conditions, trains a wind invariant representation of the unmodeled aerodynamics, and tests the performance of the model when adapting to new data in different wind conditions. 

## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
torch.set_default_tensor_type('torch.DoubleTensor')
from torch.utils.data.dataset import random_split
import os 
import utils
import mlmodel

from datetime import datetime
import re

/home/hcr/anaconda3/envs/rina/lib/python3.12/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025845206/work/torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

True
1


In [3]:
import sys
if sys.platform == 'win32':
    NUM_WORKERS = 0 # Windows does not support multiprocessing
else:
    NUM_WORKERS = 2
print('running on ' + sys.platform + ', setting ' + str(NUM_WORKERS) + ' workers')

running on linux, setting 2 workers


## Load the data and create some simple visualizations

In [4]:
# dim_a = 3
# features = ['q', 'q_dot', 'tau']
# label = 'tau_residual_m'

dim_a = 16
features = ['q', 'q_dot', 'tau_cmd']
label = 'tau_residual_cmd_centered'

labels = ["FR_hip", "FR_knee", "FR_foot", "FL_hip", "FL_knee", "FL_foot",
          "RR_hip", "RR_knee", "RR_foot", "RL_hip", "RL_knee", "RL_foot"]

# Training data collected from the neural-fly drone
dataset = 'rina' 
dataset_folder = '/home/hcr/Research/DARoSLab/DARoS-Core/lcm_converted_log/05_17_2024_formal/training_data_ex_centered/'
testdata_folder = "/home/hcr/Research/DARoSLab/DARoS-Core/lcm_converted_log/05_17_2024_formal/eval_data_ex_centered/"


# dataset_folder = '/home/oyoungquist/Research/RINA/rina/data/lcm_converted_log/05_17_2024_formal/training_data_ex/'
# testdata_folder = "/home/oyoungquist/Research/RINA/rina/data/lcm_converted_log/05_17_2024_formal/eval_data_ex/"



now = datetime.now() # current date and time
date_time = now.strftime("%m/%d/%Y%H:%M:%S")
date_time = re.sub('[^0-9a-zA-Z]+', '_', date_time)

date_time += "_cmd_residual_centered_96_128_a16_h20_e10K"

cwd = os.getcwd()

output_path_base = os.path.join(cwd, "training_results", date_time)

if not os.path.exists(output_path_base):
    os.makedirs(output_path_base)

print(output_path_base)

# # Training data collected from an intel aero drone
# dataset = 'neural-fly-transfer'
# dataset_folder = 'data/training-transfer'
# hover_pwm = 910 # mean hover pwm for neural-fly drone
# intel_hover_pwm = 1675 # mean hover pwm for intel-aero drone
# hover_pwm_ratio = hover_pwm / intel_hover_pwm # scaling ratio from system id

modelname = f"{dataset}_dim-a-{dim_a}_{'-'.join(features)}" # 'intel-aero_fa-num-Tsp_v-q-pwm'

print(modelname)

/home/hcr/Research/DARoSLab/rina/training_results/06_21_202414_01_10_cmd_residual_centered_96_128_a16_h20_e10K
rina_dim-a-16_q-q_dot-tau_cmd


In [5]:
RawData = utils.load_data(dataset_folder)
Data = utils.format_data(RawData, features=features, output=label, body_offset=0)

print("\n-----------------------------------------------\n")

RawData = utils.load_data(testdata_folder) # expnames='(baseline_)([0-9]*|no)wind'
TestData = utils.format_data(RawData, features=features, output=label, body_offset=0) # wind condition label, C, will not make sense for this data - that's okay since C is only used in the training process


-----------------------------------------------



In [6]:
# training_data_images = os.path.join(output_path_base, "training_data_images")
# test_data_images = os.path.join(output_path_base, "test_data_images")

# if not os.path.exists(training_data_images):
#     os.makedirs(training_data_images)

# if not os.path.exists(test_data_images):
#     os.makedirs(test_data_images)

# for data in Data:
#     utils.plot_subdataset(data, features, labels, os.path.join(training_data_images, "{:s}.png".format(data.meta['condition'])), title_prefix="(Training data)")

# for data in TestData:
#     utils.plot_subdataset(data, features, labels, os.path.join(test_data_images, "{:s}.png".format(data.meta['condition'])), title_prefix="(Testing Data)")

## Initialize some other hyperparameters

In [7]:
options = {}
options['dim_x'] = Data[0].X.shape[1]
options['dim_y'] = Data[0].Y.shape[1]
options['num_c'] = len(Data)
print('dims of (x, y) are', (options['dim_x'], options['dim_y']))
print('there are ' + str(options['num_c']) + ' different conditions')

dims of (x, y) are (36, 12)
there are 3 different conditions


In [8]:
# Set hyperparameters
options['features'] = features
options['dim_a'] = dim_a
options['loss_type'] = 'crossentropy-loss'

options['shuffle'] = True # True: shuffle trajectories to data points
# options['K_shot'] = 32 # number of K-shot for least square on a
options['phi_shot'] = 256 # batch size for training phi

# options['alpha'] = 0.01 # adversarial regularization loss
# options['alpha'] = 0.055 # adversarial regularization loss


# options['learning_rate'] = 5e-4
# options['learning_rate'] = 0.001497
options['frequency_h'] = 2 # how many times phi is updated between h updates, on average
# options['SN'] = 2. # maximum single layer spectral norm of phi
# options['SN'] = 4. # maximum single layer spectral norm of phi
# options['gamma'] = 10. # max 2-norm of a
options['num_epochs'] = 10000

options['K_shot'] = 50 # number of K-shot for least square on a
options['alpha'] = 0.048 # adversarial regularization loss
options['SN'] = 6. # maximum single layer spectral norm of phi
options['gamma'] = 10. # max 2-norm of a
options['learning_rate'] = 0.0009

options['phi_shot'] = 4096 # batch size for training phi


options['phi_first_out'] = 96
options['phi_second_out'] = 128
options['discrim_hidden'] = 20

options['output_path'] = output_path_base
options['device'] = device

# Dataset Generation

The adaptation dataset will be used to update $a$ in each training loop.
The training dataset will be used to train $\phi$ in each training loop.

In [9]:
# Trainset = []
# Adaptset = []
Trainloader = []
Adaptloader = []
for i in range(options['num_c']):
    fullset = mlmodel.MyDataset(Data[i].X, Data[i].Y, Data[i].C)
    
    l = len(Data[i].X)
    if options['shuffle']:
        trainset, adaptset = random_split(fullset, [int(2/3*l), l-int(2/3*l)])
    else:
        trainset = mlmodel.MyDataset(Data[i].X[:int(2/3*l)], Data[i].Y[:int(2/3*l)], Data[i].C) 
        adaptset = mlmodel.MyDataset(Data[i].X[int(2/3*l):], Data[i].Y[int(2/3*l):], Data[i].C)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=options['phi_shot'], shuffle=options['shuffle'], num_workers=NUM_WORKERS)
    adaptloader = torch.utils.data.DataLoader(adaptset, batch_size=options['K_shot'], shuffle=options['shuffle'], num_workers=NUM_WORKERS)
   
    # Trainset.append(trainset)
    # Adaptset.append(adaptset)
    Trainloader.append(trainloader) # for training phi
    Adaptloader.append(adaptloader) # for LS on a

# Domain Adversarially Invariant Meta Learning

Assume the state $x\in\mathbb{R}^n$ and $c$ is hidden state used to represent changing environment. We are interested in learning some function $f(x(t),c(t))$. $f(x(t),c(t))$ can be separated into three terms: $$f(x(t),c(t))=\phi(x(t))a(c(t))+d(t),$$
where $\phi(x(t))$ captures the $c$-variant part and $a(c(t))\in\mathbb{R}^m$ is implicitly a function of the hidden state $c(t)$. Finally, $d(t)$ is the residual noise term.

We want to learn $\phi(x)$ such that it doesn't include any information about $c$. To reach this goal, we introduce another neural network $h$ where $h(\phi(x))$ tries to predict $c$.

The loss function is given as
$$\max_h\min_{\phi, \left\{a_{c_j}\right\}_j}\sum_{j}\sum_{i}\left\|\phi(x^{(i)}_{c_j})a_{c_j}-f(x^{(i)}_{c_j},c_j)\right\|^2-\alpha\cdot\text{CrossEntropy}\left(h(\phi(x^{(i)}_{c_j})),j\right)$$
Note that the $\text{CrossEntropy-loss}$ will not require physical encoding of $c_j$ in training, only a label for $c$ that corresponds to the subdataset (that is, the label $c$ has no physical meaning).

# Initialize the models

In [10]:
# Store the model class definition in an external file so they can be referenced outside this script
phi_net = mlmodel.Phi_Net(options)
h_net = mlmodel.H_Net_CrossEntropy(options)

# push to GPU
phi_net.to(device)
h_net.to(device)

H_Net_CrossEntropy(
  (fc1): Linear(in_features=16, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=3, bias=True)
)

In [11]:
criterion = nn.MSELoss()
criterion_h = nn.CrossEntropyLoss()
optimizer_h = optim.Adam(h_net.parameters(), lr=options['learning_rate'])
optimizer_phi = optim.Adam(phi_net.parameters(), lr=options['learning_rate'])

# Meta-Training Algorithm

**Step 0: sample $c$, and sample $B+K$ data points in correponding subdataset $\{x_i,c,f(x_i,c)\}_{i}$**

**Step 1: estimate $a$ using least-square**

$K$ data points (sampled from the same wind condition $c$) are used to compute $a$ using least-squares, i.e., adaptation:
$$
\underbrace{
    \begin{bmatrix}
        \phi(x_1) \\ \phi(x_2) \\ \vdots \\ \phi(x_K) 
    \end{bmatrix}}
    _{\Phi\in\mathbb{R}^{K\times \dim(a)}}
\cdot
\underbrace{
    \begin{bmatrix}
        a_1 & \cdots & a_{\dim(y)} 
    \end{bmatrix}}
    _{a\in\mathbb{R}^{\dim(a)\times \dim(y)}}
=
\underbrace{
    \begin{bmatrix}
        f_1(x_1) & \cdots & f_{\dim(y)}(x_1) \\ f_1(x_2) & \cdots & f_{\dim(y)}(x_2) \\ \vdots & \vdots & \vdots\\ f_1(x_K) & \cdots & f_{\dim(y)}(x_K) 
    \end{bmatrix}}
    _{Y\in\mathbb{R}^{K\times \dim(y)}}
$$

The least square solution is given by
$$a=(\Phi^\top\Phi)^{-1}\Phi^\top Y$$
Normalization on $a$ is implemented to avoid ambiguity of $\phi(x)a$ (since $\phi(x)a=(0.1\phi(x))\cdot(10a)$):
$$a\leftarrow \gamma\cdot\frac{a}{\|a\|_F},\quad\text{if}\,\,\|a\|_F>\gamma$$
Note that $a$ is an implicit function of $\phi$.

**Step 2: fix $h$ and train $\phi$**

With this $a$, another $B$ data points (with same $c$) are used for gradient descent with loss
$$\mathcal{L}(\phi)=\|f(x)-\phi(x)a\|_2^2-\alpha\cdot\|h(\phi(x))-c\|_2^2$$

**Step 3: fix $\phi$ and train discriminator $h$**

Finally, these $B$ data points are used again for gradient descent on $h$ with loss
$$\mathcal{L}(h)=\|h(\phi(x))-c\|_2^2$$
We may run this step less frequently than step 2, to improve stability in training (a trick from GAN).

In [12]:
model_save_freq = 50 # How often to save the model

# Create some arrays to save training statistics
Loss_f = [] # combined force prediction loss
Loss_c = [] # combined adversarial loss

# Loss for each subdataset 
Loss_test_nominal = [] # loss without any learning
Loss_test_mean = [] # loss with mean predictor
Loss_test_phi = [] # loss with NN
for i in range(len(TestData)):
    Loss_test_nominal.append([])
    Loss_test_mean.append([])
    Loss_test_phi.append([])

# Training!
for epoch in range(options['num_epochs']):
    # Randomize the order in which we train over the subdatasets
    arr = np.arange(options['num_c'])
    np.random.shuffle(arr)

    # Running loss over all subdatasets
    running_loss_f = 0.0
    running_loss_c = 0.0

    phi_net.to(device)
    h_net.to(device)

    for i in arr:
        with torch.no_grad():
            adaptloader = Adaptloader[i]
            kshot_data = next(iter(adaptloader))
            trainloader = Trainloader[i]
            data = next(iter(trainloader))
        
        optimizer_phi.zero_grad()

        '''options['phi_shot'] = 256 # batch size for training phi

        Least-square to get $a$ from K-shot data
        '''
        # push data to device
        X = kshot_data['input'].to(device) # K x dim_x
        Y = kshot_data['output'].to(device) # K x dim_y
        Phi = phi_net(X) # K x dim_a
        Phi_T = Phi.transpose(0, 1) # dim_a x K
        A = torch.inverse(torch.mm(Phi_T, Phi)) # dim_a x dim_a
        a = torch.mm(torch.mm(A, Phi_T), Y) # dim_a x dim_y
        if torch.norm(a, 'fro') > options['gamma']:
            a = a / torch.norm(a, 'fro') * options['gamma']

        # push data off of device
        X.cpu()
        Y.cpu()
        A.cpu()
            
        '''
        Batch training \phi_net
        '''
        inputs = data['input'].to(device) # B x dim_x
        labels = data['output'].to(device) # B x dim_y
        
        c_labels = data['c'].type(torch.long).to(device)
            
        # forward + backward + optimize
        outputs = torch.mm(phi_net(inputs), a)
        loss_f = criterion(outputs, labels)
        temp = phi_net(inputs)
        
        loss_c = criterion_h(h_net(temp), c_labels)
            
        loss_phi = loss_f - options['alpha'] * loss_c
        loss_phi.backward()
        optimizer_phi.step()
        
        '''
        Discriminator training
        '''
        if np.random.rand() <= 1.0 / options['frequency_h']:
            optimizer_h.zero_grad()
            temp = phi_net(inputs)
            
            loss_c = criterion_h(h_net(temp), c_labels)
            
            loss_h = loss_c
            loss_h.backward()
            optimizer_h.step()
        
        '''
        Spectral normalization
        '''
        # push network to CPU for normalization
        phi_net.to('cpu')
        if options['SN'] > 0:
            for param in phi_net.parameters():
                M = param.detach().numpy()
                if M.ndim > 1:
                    s = np.linalg.norm(M, 2)
                    if s > options['SN']:
                        param.data = param / s * options['SN']
         
        # push network to device
        phi_net.to(device)
        
        running_loss_f += loss_f.cpu().item()
        running_loss_c += loss_c.cpu().item()

        # push data back to cpu
        inputs.to('cpu')
        labels.to('cpu')
        c_labels.to('cpu')
        a.to('cpu')
    
    # Save statistics
    Loss_f.append(running_loss_f / options['num_c'])
    Loss_c.append(running_loss_c / options['num_c'])
    if epoch % 10 == 0:
        print('[%d] loss_f: %.2f loss_c: %.2f' % (epoch + 1, running_loss_f / options['num_c'], running_loss_c / options['num_c']))
        
    with torch.no_grad():
        for j in range(len(TestData)):
            loss_nominal, loss_mean, loss_phi = mlmodel.error_statistics(TestData[j].X, TestData[j].Y, phi_net, h_net, options=options)
            Loss_test_nominal[j].append(loss_nominal)
            Loss_test_mean[j].append(loss_mean)
            Loss_test_phi[j].append(loss_phi)

    if epoch % model_save_freq == 0:
        mlmodel.save_model(phi_net=phi_net, h_net=h_net, modelname=modelname + '-epoch-' + str(epoch), options=options)

mlmodel.save_model(phi_net=phi_net, h_net=h_net, modelname=modelname + '-epoch-' + str(options["num_epochs"]), options=options)

<>:57: SyntaxWarning: invalid escape sequence '\p'
<>:57: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_143671/2284369876.py:57: SyntaxWarning: invalid escape sequence '\p'
  '''


[1] loss_f: 7.10 loss_c: 1.10
[11] loss_f: 6.64 loss_c: 1.10
[21] loss_f: 6.03 loss_c: 1.10
[31] loss_f: 5.24 loss_c: 1.10
[41] loss_f: 5.28 loss_c: 1.10
[51] loss_f: 4.56 loss_c: 1.11
[61] loss_f: 5.06 loss_c: 1.10
[71] loss_f: 5.06 loss_c: 1.11
[81] loss_f: 3.99 loss_c: 1.11
[91] loss_f: 5.11 loss_c: 1.10
[101] loss_f: 4.99 loss_c: 1.10
[111] loss_f: 3.92 loss_c: 1.10
[121] loss_f: 4.14 loss_c: 1.09
[131] loss_f: 4.49 loss_c: 1.09
[141] loss_f: 3.84 loss_c: 1.09
[151] loss_f: 3.81 loss_c: 1.09
[161] loss_f: 3.61 loss_c: 1.09
[171] loss_f: 4.05 loss_c: 1.09
[181] loss_f: 3.72 loss_c: 1.09
[191] loss_f: 3.48 loss_c: 1.09
[201] loss_f: 3.51 loss_c: 1.10
[211] loss_f: 3.78 loss_c: 1.10
[221] loss_f: 3.52 loss_c: 1.10


Traceback (most recent call last):
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/shutil.py", line 784, in rmtree
    onexc(os.rmdir, path, err)
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/shutil.py", line 782, in rmtree
    os.rmdir(path, dir_fd=dir_fd)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-t17y2a76'


[231] loss_f: 3.24 loss_c: 1.10
[241] loss_f: 3.69 loss_c: 1.09
[251] loss_f: 3.16 loss_c: 1.10
[261] loss_f: 4.36 loss_c: 1.10
[271] loss_f: 3.92 loss_c: 1.09
[281] loss_f: 3.66 loss_c: 1.09
[291] loss_f: 3.17 loss_c: 1.09
[301] loss_f: 3.48 loss_c: 1.09
[311] loss_f: 3.02 loss_c: 1.09
[321] loss_f: 3.02 loss_c: 1.09


Traceback (most recent call last):
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/shutil.py", line 784, in rmtree
    onexc(os.rmdir, path, err)
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/shutil.py", line 782, in rmtree
    os.rmdir(path, dir_fd=dir_fd)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-mb6s6i8k'
Traceback (most recent call last):
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/hcr/anaconda3/envs/rina/lib/pytho

...

[1301] loss_f: 2.63 loss_c: 1.07
[1311] loss_f: 2.71 loss_c: 1.07
[1321] loss_f: 2.74 loss_c: 1.07
[1331] loss_f: 2.49 loss_c: 1.07
[1341] loss_f: 2.66 loss_c: 1.08
[1351] loss_f: 2.30 loss_c: 1.08


Traceback (most recent call last):
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/shutil.py", line 784, in rmtree
    onexc(os.rmdir, path, err)
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/shutil.py", line 782, in rmtree
    os.rmdir(path, dir_fd=dir_fd)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-p00t5x3x'


[1361] loss_f: 2.95 loss_c: 1.08
[1371] loss_f: 2.77 loss_c: 1.08
[1381] loss_f: 2.41 loss_c: 1.06
[1391] loss_f: 2.77 loss_c: 1.06
[1401] loss_f: 2.33 loss_c: 1.06


Exception ignored in: <function _releaseLock at 0x7fc512da4860>
Traceback (most recent call last):
  File "/home/hcr/anaconda3/envs/rina/lib/python3.12/logging/__init__.py", line 243, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


[1411] loss_f: 2.46 loss_c: 1.06


In [ ]:
plt.subplot(2, 1, 1)
plt.plot(Loss_f)
plt.xlabel('epoch')
plt.ylabel('f-loss [N]')
plt.title('training f loss')
plt.subplot(2, 1, 2)
plt.plot(Loss_c)
plt.title('training c loss')
plt.xlabel('epoch')
plt.ylabel('c-loss')
plt.tight_layout()

for j in range(len(TestData)):
    plt.figure()
    # plt.plot(Loss_test_nominal[j], label='nominal')
    plt.plot(Loss_test_mean[j], label='mean')
    plt.plot(np.array(Loss_test_phi[j]), label='phi*a')
    # plt.plot(np.array(Loss_test_exp_forgetting[j]), label='exp forgetting')
    plt.legend()
    plt.title(f'Test data set {j} - {TestData[j].meta["condition"]}')

plt.show()

In [ ]:
# Choose final model
# stopping_epoch = 200
# options['num_epochs'] = stopping_epoch

# model_path = os.path.join(options["output_path"], "models", (modelname + '-epoch-' + str(options["num_epochs"])))
model_path = os.path.join("/home/hcr/Research/DARoSLab/rina/training_results/06_19_202421_28_14_cmd_residual_96_128_a16_h20_e10K/models", (modelname + '-epoch-' + str(2000)))
final_model = mlmodel.load_model(modelname = model_path)

In [ ]:
phi_net = final_model.phi
h_net = final_model.h

eval_adapt_start = 0
eval_adapt_end = 2500
eval_val_start = 2500
eval_val_end = 5000

vis_output_path_prefix_training_data = os.path.join(options["output_path"], "eval_images", "training")
vis_output_path_prefix_testing_data = os.path.join(options["output_path"], "eval_images", "testing")

# vis_output_path_prefix_training_data = os.path.join("/home/hcr/Research/DARoSLab/rina/training_results/04_02_202415_31_57/eval_images/training")
# vis_output_path_prefix_testing_data = os.path.join("/home/hcr/Research/DARoSLab/rina/training_results/04_02_202415_31_57/eval_images/testing")

Plot the measured aerodynamic force, labeled ground truth (gt), along with the region used for adapation (adapt), and the predicted region (val)

In [ ]:
phi_net.to(device)

for i, data in enumerate(Data):
    print('------------------------------')
    print(data.meta['condition'] + ':')
    file_name = "{:s}.png".format(data.meta['condition'])
    mlmodel.vis_validation(t=data.meta['steps'], x=data.X, y=data.Y, phi_net=phi_net, 
                           h_net=h_net, idx_adapt_start=eval_adapt_start, idx_adapt_end=eval_adapt_end, 
                           idx_val_start=eval_val_start, idx_val_end=eval_val_end, c=Data[i].C, options=options, 
                           output_path_prefix=vis_output_path_prefix_training_data, output_name=file_name)

In [ ]:
for data in Data:
    image_name = "{:s}_errors_hist.png".format(data.meta['condition'])
    error_1, error_2, error_3 = mlmodel.error_statistics_hist(data.X, data.Y, phi_net, h_net, options, vis_output_path_prefix_training_data, image_name)
    print('**** c =', str(data.C), ':', data.meta['condition'], '****')
    print(f'Before learning: MSE is {error_1: .2f}')
    print(f'Mean predictor: MSE is {error_2: .2f}')
    print(f'After learning phi(x): MSE is {error_3: .2f}')
    print('')

## Test Data Error Analysis

In [ ]:
for i, data in enumerate(TestData):
    print('------------------------------')
    print(data.meta['condition'] + ':')
    print(len(data.X))
    file_name = "{:s}.png".format(data.meta['condition'])
    mlmodel.vis_validation(t=data.meta['steps'], x=data.X, y=data.Y, phi_net=phi_net, h_net=h_net, 
                           idx_adapt_start=eval_adapt_start, idx_adapt_end=eval_adapt_end, 
                           idx_val_start=eval_val_start, idx_val_end=eval_val_end, c=TestData[i].C, options=options,
                           output_path_prefix=vis_output_path_prefix_testing_data, output_name=file_name)

In [ ]:
for data in TestData:
    image_name = "{:s}_errors_hist.png".format(data.meta['condition'])
    error_1, error_2, error_3 = mlmodel.error_statistics_hist(data.X, data.Y, phi_net, h_net, options, vis_output_path_prefix_testing_data, image_name)
    print('**** :', data.meta['condition'], '****')
    print(f'Before learning: MSE is {error_1: .2f}')
    print(f'Mean predictor: MSE is {error_2: .2f}')
    print(f'After learning phi(x): MSE is {error_3: .2f}')
    print('')

In [ ]:
print(final_model.phi.options['device'])
final_model.phi.options['device'] = 'cpu'
final_model.phi.options['device']

In [ ]:
final_model.phi.to('cpu')

# convert the trained python model to a Torch.Script model
# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 36).to('cpu')

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(final_model.phi, example)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(final_model.phi, example)

# testing the traced output
print(traced_script_module(example))

traced_script_module.cpu()

# save-out the scripted model _cmd_residual_96_128_h20_e10K
traced_script_module.save("traced_rina_model_cmd_error_centered_a16_96_128_h20_e10K.pt")